In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template("{num} 의 10배는?")
chain = prompt | ChatOpenAI(model="gpt-4o")

In [8]:
chain.invoke({"num": 5})

AIMessage(content='5의 10배는 50입니다.', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 14, 'total_tokens': 24}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_729ea513f7', 'finish_reason': 'stop', 'logprobs': None}, id='run-8018088b-431c-466f-852f-c08ae26dbefa-0')

In [11]:
# RunnablePassthrough : 입력을 변경하지 않거나 추가 키를 더하여 전달

from langchain_core.runnables import RunnablePassthrough

runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI(model="gpt-4o")
runnable_chain.invoke({"num": 5})

AIMessage(content="{'num': 5}의 10배는 50입니다. 이를 계산하기 위해서 'num' 값을 10과 곱하면 됩니다.\n\n```python\ndata = {'num': 5}\nresult = data['num'] * 10\nprint(result)  # 출력: 50\n```\n\n따라서 50이 정답입니다.", response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 19, 'total_tokens': 94}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_729ea513f7', 'finish_reason': 'stop', 'logprobs': None}, id='run-b6f92501-a271-420a-af7f-26acf9ba9e66-0')

In [12]:
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 5})

{'num': 5, 'new_num': 15}

In [13]:
# RunnableParallel : 여러 Runnable을 병렬로 실행

from langchain_core.runnables import RunnableParallel

runnable = RunnableParallel(
    runnable=RunnableParallel(
        passed=RunnablePassthrough(),
        extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
        modified=lambda x: x["num"] + 1,
    ),
)

runnable.invoke({"num": 5})

{'runnable': {'passed': {'num': 5},
  'extra': {'num': 5, 'mult': 15},
  'modified': 6}}

In [16]:
from langchain_core.runnables import RunnableLambda

def extra(x):
    ext = x['runnable']['extra']  # 'extra': {'num': 5, 'mult': 15}
    return int(ext["num"]) * int(ext["mult"])  # 5 * 15

(runnable | RunnableLambda(extra)).invoke({"num": 5})

75